In [2]:
import os
import urllib.request

import numpy as np
import pytest
import scanpy as sc
import torch
from scdataloader import Preprocessor
from scdataloader.utils import populate_my_ontology

from scprint import scPrint
from scprint.base import NAME
from scprint.tasks import Denoiser, Embedder, GNInfer

%reload_ext autoreload
%autoreload 2


In [19]:
adata = sc.read_h5ad("test.h5ad")
adata.obs.drop(columns="is_primary_data", inplace=True, errors="ignore")
adata.obs["organism_ontology_term_id"] = "NCBITaxon:9606"
preprocessor = Preprocessor(
    do_postp=False,
    force_preprocess=True,
)
adata = preprocessor(adata)

ckpt_path = "small.ckpt"
model = scPrint.load_from_checkpoint(
    ckpt_path,
    precpt_gene_emb=None,
    # triton gets installed so it must think it has cuda enabled
    transformer="normal",
)

Dropping layers:  KeysView(Layers with keys: )
checking raw counts
removed 0 non primary cells, 1000 renamining
filtered out 0 cells, 1000 renamining
Removed 0 genes.
validating


/home/ml4ig1/Documents code/scPRINT/.venv/lib/python3.10/site-packages/scdataloader/preprocess.py:248: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data_utils.validate(adata, organism=adata.obs.organism_ontology_term_id[0])
/home/ml4ig1/Documents code/scPRINT/.venv/lib/python3.10/site-packages/lamindb/_can_validate.py:113: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if isinstance(values[0], list):
/home/ml4ig1/Documents code/scPRINT/.venv/lib/python3.10/site-packages/lamindb/_can_validate.py:113: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will alwa

startin QC
Seeing 161 outliers (16.10% of total dataset):
done
RuntimeError caught: scPrint is not attached to a `Trainer`.


In [20]:
adata

View of AnnData object with n_obs × n_vars = 1000 × 70611
    obs: 'biosample_id', 'donor_id', 'cell_type_ontology_term_id', 'organism_ontology_term_id', 'disease_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'cell_type__custom', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'age', 'self_reported_ethnicity_ontology_term_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'cell_culture', 'nnz', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'uid', 'symbol', 'ncbi_gene_ids', 'biotype', 'descr

In [21]:
cell_embedder = Embedder(
    batch_size=6,
    num_workers=1,
    how="random expr",
    max_len=300,
    doclass=True,
    pred_embedding=[
        "cell_type_ontology_term_id",
        "disease_ontology_term_id",
        "self_reported_ethnicity_ontology_term_id",
        "sex_ontology_term_id",
    ],
    plot_corr_size=10,
    doplot=True,
    keep_all_cls_pred=False,
    dtype=torch.float32,
)
adata_emb, metrics = cell_embedder(model, adata[:10, :])

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


View of AnnData object with n_obs × n_vars = 10 × 70611
    obs: 'biosample_id', 'donor_id', 'cell_type_ontology_term_id', 'organism_ontology_term_id', 'disease_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'cell_type__custom', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'age', 'self_reported_ethnicity_ontology_term_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'cell_culture', 'nnz', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes'
    var: 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype', 'uid', 'symbol', 'ncbi_gene_ids', 'biotype', 'descrip

 50%|█████     | 1/2 [00:00<00:00,  7.37it/s]

hey
hey


100%|██████████| 2/2 [00:00<00:00,  8.71it/s]


AnnData object with n_obs × n_vars = 10 × 128
    obs: 'pred_cell_type_ontology_term_id', 'pred_disease_ontology_term_id', 'pred_assay_ontology_term_id', 'pred_self_reported_ethnicity_ontology_term_id', 'pred_sex_ontology_term_id', 'pred_organism_ontology_term_id', 'conv_pred_cell_type_ontology_term_id', 'conv_pred_disease_ontology_term_id', 'conv_pred_assay_ontology_term_id', 'conv_pred_self_reported_ethnicity_ontology_term_id'
couldn't log to tensorboard
couldn't log to wandb
AnnData object with n_obs × n_vars = 10 × 128
    obs: 'pred_cell_type_ontology_term_id', 'pred_disease_ontology_term_id', 'pred_assay_ontology_term_id', 'pred_self_reported_ethnicity_ontology_term_id', 'pred_sex_ontology_term_id', 'pred_organism_ontology_term_id', 'conv_pred_cell_type_ontology_term_id', 'conv_pred_disease_ontology_term_id', 'conv_pred_assay_ontology_term_id', 'conv_pred_self_reported_ethnicity_ontology_term_id'
too few cells to embed into a umap
too few cells to compute a clustering
     cell_t

In [22]:
adata_emb

AnnData object with n_obs × n_vars = 10 × 70611
    obs: 'biosample_id', 'donor_id', 'cell_type_ontology_term_id', 'organism_ontology_term_id', 'disease_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'cell_type__custom', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'age', 'self_reported_ethnicity_ontology_term_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'cell_culture', 'nnz', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes', 'pred_cell_type_ontology_term_id', 'pred_disease_ontology_term_id', 'pred_assay_ontology_term_id', 'pred_self_reported_ethnicity_ontology_term_id', 

In [24]:
assert any(
    col.startswith("pred_") for col in adata_emb.obs.columns
), "Classification failed"

In [25]:
grn_inferer = GNInfer(
    layer=[0, 1],
    batch_size=2,
    how="random expr",
    preprocess="softmax",
    head_agg="mean",
    filtration="none",
    forward_mode="none",
    num_genes=100,
    max_cells=10,
    doplot=False,
)
grn_adata = grn_inferer(model, adata)

100%|██████████| 5/5 [00:00<00:00,  7.66it/s]


avg link count: 85969984, sparsity: 1.0


In [27]:
assert "GRN" in grn_adata.varp, "GRN inference failed"